# RNN Based molucule generation

Laurent Cetinsoy

In this hands-on we want to generate molecule formulas for denovo-drug discovery.

For that we need to use Generative models. Generative models are models which goes beyond classification or simple regression : they are able to generate data that look like previously seens dataset.

There exists a lot of models :

- Bayesian models like graphical models
- Recurrent models (for sequence generation like texte)
- Variational auto encoders
- Generative adversarial models
- Flow and diffusion models


In the hands-on we will start by  trainning a character based RNN to generate smile molecules


We want to feed smile representations of molecules to an RNN.
The basic idea is we will train it to predict the next smile token of a molecule given the previous one.

For instance for the following molecule "CC(=O)NC1=CC=C(O)C=C1" will may give to the model

X = "CC(=O)N"
y = C

and ask the RNN to learn to predict y given X

Like a standard language model !


## RNN Language model


A language model is a model which predict the next token of a sequence given the previous ones :

$ P(X_t | X_{t-1}, X_{t-2}, ..., X_{t-p})  $


This model can be learned with a Recurrent neural network

$ y = P(X_t | X_{t-1}, X_{t-2}, ..., X_{t-p}) = RNN_{\theta} (X_{t-1}, X_{t-2}, ..., X_{t-p})  $


In order to train such model you need a corpus of data.



There are two main ways to do that : Word level model or character level model

For character level models, an interesting resource is : http://karpathy.github.io/2015/05/21/rnn-effectiveness/



Explain briefly what is the difference between word based language model and character based language model

## Loading the data

Dowload the following dataset : https://github.com/joeymach/Leveraging-VAE-to-generate-molecules

In [2]:
import pandas as pd
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import ModelCheckpoint
import os
import numpy as np
from keras.models import Sequential



Import pandas and load the first 1000 lines

In [5]:
data_path = '250k_smiles.csv'  # Remplacer par le chemin de votre dataset
df = pd.read_csv(data_path).head(1000)  # Charger les 1000 premières lignes

Display the first rows of the dataframe

In [6]:
df.head()

,smiles,logP,qed,SAS
0,CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1\n,5.05060,0.702012,2.084095
1,C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1\n,3.11370,0.928975,3.432004
2,N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)...,4.96778,0.599682,2.470633
3,CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c...,4.00022,0.690944,2.822753
4,N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#...,3.60956,0.789027,4.035182


## Processing the data

We need to do the following things :

- convert smile tokens to numbers
- build  smile token sequences and corresponding labels pairs

Compute the biggest smile molecule size

In [8]:
longest_length = df['smiles'].str.len().max()
print(f"The bigest molecule size in smiles is: {longest_length}")

The bigest molecule size in smiles is: 106



Code a function **unic_characters(string)** which return the unic characters in a string


In [9]:
def unic_characters(text):
    """Renvoie une liste triée des caractères uniques dans un texte donné."""
    return sorted(set(text))
#test
unic_characters("bbbbbooooojjjjjrsssss")


['b', 'j', 'o', 'r', 's']

Concatenate all smile string of the pandas dataframe and use **unic_characters** to get the unic_characters

In [11]:
# Concaténer toutes les chaînes SMILES
all_molecules = ''.join(df['smiles'].tolist())
distinct_chars = unic_characters(all_molecules)
distinct_chars

['\n',
 '#',
 '(',
 ')',
 '+',
 '-',
 '/',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '=',
 '@',
 'B',
 'C',
 'F',
 'H',
 'I',
 'N',
 'O',
 'S',
 '[',
 '\\',
 ']',
 'c',
 'l',
 'n',
 'o',
 'r',
 's']

Code a function **map_char_to_int(unic_chars)** which returns a dictionnary where each char is assigned an int value.
Add a character to specify the end of the molecule (like "\n")


In [12]:
# prompt: Code a function map_char_to_int(unic_chars) which returns a dictionnary where each char is assigned an int value. Add a character to specify the end of the molecule (like "\n")

def map_char_to_int(unic_chars):
    """
    Maps unique characters to integer values.

    Args:
      unic_chars: A list of unique characters.

    Returns:
      A dictionary where keys are characters and values are integer mappings.
    """
    char_to_int = dict((c, i) for i, c in enumerate(unic_chars))
    char_to_int['\n'] = len(char_to_int)  # Add end-of-molecule character
    return char_to_int

Code a function map_int_to_char(unic_chars) which returns the reverse mapping.

If you want you can merge both functions in a class

In [13]:
# prompt: Code a function map_int_to_char(unic_chars) which returns the reverse mapping.
# If you want you can merge both functions in a class

def map_int_to_char(unic_chars):
    """
    Maps integer values back to unique characters.

    Args:
      unic_chars: A list of unique characters.

    Returns:
      A dictionary where keys are integer mappings and values are characters.
    """
    int_to_char = dict((i, c) for i, c in enumerate(unic_chars))
    int_to_char[len(int_to_char)] = '\n'  # Add end-of-molecule character mapping
    return int_to_char

For each smile molecule add the ending token to it

In [14]:
# prompt: For each smile molecule add the ending token to it

# For each smile molecule add the ending token to it
df['smiles'] = df['smiles'] + '\n'

In [15]:
df['smiles'].head()

,smiles
0,CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1\n\n
1,C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1\n\n
2,N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)...
3,CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c...
4,N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#...


## Building the dataset

Now we will create the dataset so that it has the good share for our Keras LSTM model

Remember Keras recurrent models expect a 3D array with shapes (n_examples, seq_len, n_features)



What will be n_features in our case ?

*Code* a function **build_X_and_y(string, i_char, seq_lenght)** which takes a string, a **seq_length** number and a position.


It should create X by by getting all character between i and i + seq_length
and create y by getting the character following the X sequence
it returns X and y

Test your function on the following string "" with seq_length = 4 and i = [1, 2, 3]

In [16]:
# prompt: Now we will create the dataset so that it has the good share for our Keras LSTM model
# Remember Keras recurrent models expect a 3D array with shapes (n_examples, seq_len, n_features)
# What will be n_features in our case ?
# Double-cliquez (ou appuyez sur Entrée) pour modifier
# Code a function build_X_and_y(string, i_char, seq_lenght) which takes a string, a seq_length number and a position.
# It should create X by by getting all character between i and i + seq_length and create y by getting the character following the X sequence it returns X and y
# Test your function on the following string "" with seq_length = 4 and i = [1, 2, 3]

def build_X_and_y(string, i_char, seq_length):
    """
    Builds input (X) and output (y) sequences for an LSTM model.

    Args:
        string: The input string.
        i_char: The starting index for the sequence.
        seq_length: The length of the input sequence.

    Returns:
        A tuple containing the input sequence (X) and the corresponding output character (y).
    """
    X = string[i_char:i_char + seq_length]
    y = string[i_char + seq_length]
    return X, y

# Example usage (replace "" with your desired string)
test_string = "abcdefghij"
seq_length = 4
indices = [1, 2, 3]

for i in indices:
    X, y = build_X_and_y(test_string, i, seq_length)
    print(f"For i = {i}: X = '{X}', y = '{y}'")

For i = 1: X = 'bcde', y = 'f'
For i = 2: X = 'cdef', y = 'g'
For i = 3: X = 'defg', y = 'h'


By using build_X_and_y and map_char_to_int build a list nameed X_train and a list named y_train

In [82]:
# prompt: By using build_X_and_y and map_char_to_int build a list nameed X_train and a list named y_train

X_train = []
y_train = []
seq_length = 10

char_to_int_map = map_char_to_int(distinct_chars)

for molecule in df['smiles']:
    for i in range(0, len(molecule) - seq_length, 1):
        X, y = build_X_and_y(molecule, i, seq_length)
        X_train.append([char_to_int_map[char] for char in X])
        y_train.append(char_to_int_map[y])

Create numpy arrays from the lists

In [89]:
# prompt: Create numpy arrays from the lists

import numpy as np

X_train = np.array(X_train)
y_train = np.array(y_train)

In [90]:
X_train.shape

(36242, 10)

Reshape the X numpy array (n_examples, seq_lenght, 1)

In [96]:
# prompt: Reshape the X numpy array (n_examples, seq_lenght, 1)
print("Shape X_train before is : ",X_train.shape)
X_train_reshaped = X_train.reshape(X_train.shape[0], seq_length, 1)
print("Shape X_train after is : ",X_train_reshaped.shape)

Shape X_train before is :  (36242, 10)
Shape X_train after is :  (36242, 10, 1)


Normalize X by dividing each values by the total number of unic characters

In [99]:
# prompt: Normalize X by dividing each values by the total number of unic characters
X_train_normalized = X_train_reshaped.astype('float32') / len(distinct_chars)
print(X_train_normalized.shape)

(36242, 10, 1)


Import Keras and build (at least) a two layered LSTM network with 128 neurone in each.

You can also add Dropoutlayers

Do you think you should use the return_sequences = True ? If yes, when ?


Add a Dense layer on top with with the appropriate activation function and number of neurones


In [100]:
# prompt: Import Keras and build (at least) a two layered LSTM network with 128 neurone in each.
# You can also add Dropoutlayers
# Do you think you should use the return_sequences = True ? If yes, when ?
# Add a Dense layer on top with with the appropriate activation function and number of neurones

from keras.layers import LSTM, Dense, Dropout
from keras.models import Sequential

model = Sequential()
model.add(LSTM(128, input_shape=(10, 1), return_sequences=True)) # return_sequences=True is needed because we have another LSTM layer after this one.
model.add(Dropout(0.2))
model.add(LSTM(128)) # return_sequences is False by default for the last LSTM layer
model.add(Dropout(0.2))
model.add(Dense(len(distinct_chars), activation='softmax')) # Output layer with softmax activation


# Print model summary
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_12 (LSTM)                       │ (None, 10, 128)             │          66,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 10, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_13 (LSTM)                       │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 33)                  │           4,257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 202,401 (790.63 KB)

 Trainable params: 202,401 (790.63 KB)

 Non-trainable params: 0 (0.00 B)

Compile the model with the appropriate loss function and the adam optimizer

In [101]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')


Train the model on 20 epochs and 10 examples (yeah you read correctly) and

---

check that the model overfits !

In [102]:
# prompt: Train the model on 20 epochs and 10 examples (yeah you read correctly) and
# check that the model overfits !

# Train the model on 20 epochs and 10 examples
model.fit(X_train[:10], y_train[:10], epochs=20)

Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 3.3980
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 3.0805
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 2.8127
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 2.5409
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 2.2950
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 2.0552
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 1.7967
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 1.8256
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 1.7241
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 1.7927
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 1.6245
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 1.4281
Epoch 13/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 1.4644
Epoch 14/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 1.6603
Epoch 15/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 1.6404
Epoch 16/20
1/1 ━━━━━━━━━━━━━━━━━━━━

If it does not overfit try to fix data prep and model architecture so it does

In [66]:
#it did not overfit

Create a function **make_prediction(seed_start)** which takes a starting string sequence and uses it to generate a molecule


In [103]:
# prompt: Create a function make_prediction(seed_start) which takes a starting string sequence and uses it to generate a molecule

def make_prediction(seed_start):
    int_to_char_map = map_int_to_char(distinct_chars)
    char_to_int_map = map_char_to_int(distinct_chars)
    string_mapped = [char_to_int_map[char] for char in seed_start]
    full_string = seed_start
    #print(string_mapped)

    for i in range(longest_length):
        x = np.reshape(np.array(string_mapped), (1, len(string_mapped), 1))
        x = x.astype('float32') / len(distinct_chars)
        #print(x)
        pred_index = np.argmax(model.predict(x, verbose=0))
        seq = int_to_char_map[pred_index]
        full_string = full_string + seq
        string_mapped.append(pred_index)
        string_mapped = string_mapped[1:]
        if seq == '\n':
            break
    return full_string

generate a molecule of your overfitted model

In [104]:
# prompt: generate a molecule of your overfitted model

print(make_prediction("c"))

ccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc


Make a model checkpoint so that the model is saved after each epoch
if you train on a plateform and it stops you do not lose your training

In [105]:
# prompt: Make a model checkpoint so that the model is saved after each epoch if you train on a plateform and it stops you do not lose your training

import pandas as pd
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import ModelCheckpoint
import os
import numpy as np
from keras.models import Sequential

# ... (Your existing code) ...

# Define the filepath for model checkpoints
filepath = "model_checkpoints/weights-improvement-{epoch:02d}-{loss:.4f}.keras"

# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(filepath), exist_ok=True)

# Define the ModelCheckpoint callback
checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',  # Monitor the loss
    verbose=1,
    save_best_only=True,  # Save only the best model
    mode='min'  # Save when loss is minimized
)


# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

# Train the model with the checkpoint callback
model.fit(X_train[:10], y_train[:10], epochs=20, callbacks=[checkpoint])



Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 1.2949
Epoch 1: loss improved from inf to 1.29488, saving model to model_checkpoints/weights-improvement-01-1.2949.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 1.2949
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 1.3756
Epoch 2: loss did not improve from 1.29488
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 1.3756
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 1.3868
Epoch 3: loss did not improve from 1.29488
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 1.3868
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 1.5746
Epoch 4: loss did not improve from 1.29488
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 1.5746
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 1.3396
Epoch 5: loss did not improve from 1.29488
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 1.3396
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 1.3514
Epoch 6: loss did not improve from 1.29488
1/1 ━━━━━━━━━━━━━━━━━━━━ 0

Now go to your favorite plateform (colab or something else) and train the dataset on the whole data for 10 epochs and batch size 256

it should take a long time so either follow the class or go take a nap

In [106]:
# prompt: verify if all X_train and y_train are clean or if they contains nan value or something wrong

import pandas as pd
import numpy as np

# Assuming X_train and y_train are defined as in the provided code

# Check for NaN values in X_train
nan_in_X = np.isnan(X_train).any()
if nan_in_X:
    print("X_train contains NaN values.")
    # Optionally, handle the NaN values (e.g., imputation)
    # X_train = np.nan_to_num(X_train) # Example: Replace NaN with 0

# Check for NaN values in y_train
nan_in_y = np.isnan(y_train).any()
if nan_in_y:
    print("y_train contains NaN values.")
    # Handle NaN values in y_train if necessary

# Check for infinite values in X_train
inf_in_X = np.isinf(X_train).any()
if inf_in_X:
    print("X_train contains infinite values.")
    # Handle infinite values in X_train if necessary

# Check for infinite values in y_train
inf_in_y = np.isinf(y_train).any()
if inf_in_y:
    print("y_train contains infinite values.")
    # Handle infinite values in y_train if necessary

# Additional checks (if needed)
# Check data types, shapes, ranges, etc.

# Example of checking data types
print(f"Data type of X_train: {X_train.dtype}")
print(f"Data type of y_train: {y_train.dtype}")

# Example of checking shapes
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")

Data type of X_train: int64
Data type of y_train: int64
Shape of X_train: (36242, 10)
Shape of y_train: (36242,)


In [110]:
from tensorflow.keras.utils import to_categorical

y_train_encoded = to_categorical(y_train, num_classes=34)  # 34 classes


model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(X_train_normalized.shape[1], 1)),
    LSTM(64),
    Dense(34, activation='softmax')  # Assurez-vous que le nombre de sorties correspond aux classes (34 dans ce cas)
])

# Compilation du modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrainer le modèle sur 10 epochs avec un batch_size de 256
history = model.fit(X_train_normalized, y_train_encoded,
                    epochs=10,
                    batch_size=256,
                    validation_split=0.1,
                    callbacks=[checkpoint],
                    verbose=1)

Epoch 1/10
125/128 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1814 - loss: 2.9490
Epoch 1: loss did not improve from 0.82703
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.1816 - loss: 2.9437 - val_accuracy: 0.1686 - val_loss: 2.7209
Epoch 2/10
121/128 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1956 - loss: 2.6875
Epoch 2: loss did not improve from 0.82703
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.1955 - loss: 2.6879 - val_accuracy: 0.1975 - val_loss: 2.7006
Epoch 3/10
126/128 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2019 - loss: 2.6743
Epoch 3: loss did not improve from 0.82703
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.2019 - loss: 2.6741 - val_accuracy: 0.1848 - val_loss: 2.6461
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2159 - loss: 2.6019
Epoch 4: loss did not improve from 0.82703
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.2159 - loss: 2.6017 - val_accuracy: 0.2472 - val_loss: 2.5065
Epoch 5/10
128/1

Generate between 100 and 1000 molecules.

create a list where molecules have between 10 and 50 atoms

In [116]:
import numpy as np

# Function to generate multiple molecules
def generate_molecules(num_molecules=1000, min_atoms=10, max_atoms=50):
    generated_molecules = []

    while len(generated_molecules) < num_molecules:
        seed = "CC"  # Starting seed
        molecule = make_prediction(seed)

        # Check if molecule length (excluding padding) is within the desired range
        if min_atoms <= len(molecule.strip()) <= max_atoms:
            generated_molecules.append(molecule.strip())  # Remove padding and add to the list

    return generated_molecules

# Generate molecules
molecules = generate_molecules(1000, 10, 50)

# Output the number of molecules generated
print(f"Total molecules generated: {len(molecules)}")


KeyboardInterrupt: 

[texte du lien](https://)With rdkit compute the Quantified Estimated Drug likelyness (QED) of each molecule in this subset

In [ ]:
def compute_qed(molecules):
    qed_values = []

    for mol in molecules:
        # Convert the SMILES string to an RDKit molecule object
        mol_obj = Chem.MolFromSmiles(mol)
        if mol_obj is not None:
            # Compute the QED
            qed = QED.qed(mol_obj)
            qed_values.append(qed)
        else:
            qed_values.append(None)  # If molecule is invalid or cannot be processed, return None

    return qed_values

molecules = generate_molecules(1000, 10, 50)

qed_values = compute_qed(molecules)

print(f"Total molecules generated: {len(molecules)}")
print(f"QED values computed: {len(qed_values)}")
print(f"Mean QED: {np.mean([q for q in qed_values if q is not None]):.3f}")
print(f"Median QED: {np.median([q for q in qed_values if q is not None]):.3f}")
print(f"Number of molecules with valid QED values: {len([q for q in qed_values if q is not None])}")

# Optionally, print some QED values for inspection
print("Sample QED values:")
for mol, qed in zip(molecules[:10], qed_values[:10]):
    print(f"Molecule: {mol[:30]}... QED: {qed:.3f}")


Bonus 1 : Using rdkit, compute the quantitative estimation of drug-likeness (QED) of your generated molecules.

Bonus 2 : try to adapt a transformer model training from hugging face to see if it is better